In [1]:
pip install psaw

Note: you may need to restart the kernel to use updated packages.


In [1]:
from psaw import PushshiftAPI
import requests
from ast import literal_eval
import datetime as dt

In [2]:
api = PushshiftAPI()

In [67]:
def generate_data(sub):
    total_entries = []
    start_epoch=int(dt.datetime(2019, 10, 1).timestamp())
    gen = api.search_submissions(after=start_epoch,subreddit=sub,sort_type='num_comments')

    max_response_cache = 2000
    cache = []
    
    #Get 2000 posts from a certain subreddit after October 1st 2019 sorted by most comments to least
    for c in gen:
        cache.append(c)
        if len(cache) >= max_response_cache:
            break

    for i in range(len(cache)):
        title = cache[i].title
        selftext = cache[i].selftext
        if selftext == '':
            selftext = cache[i].url
        #Stop getting posts if there are few comments in remaining posts
        if cache[i].num_comments < 5:
            break
        #Do not include posts that have been removed or deleted
        elif selftext != '[deleted]' and selftext != '[removed]':
            #Get List of comment ids from a post
            response = requests.get("https://api.pushshift.io/reddit/submission/comment_ids/"+cache[i].id)
            comment_ids = literal_eval(response.text)['data']
            comment_gen = api.search_comments(ids=comment_ids)
            comments = []
            #Add comment to list of post comments if it has not been removed, deleted or came from a bot
            for c in comment_gen:
                if c.parent_id.split('_')[1] == cache[i].id:
                    if (c.body != '[deleted]') and (c.body != '[removed]') and ('I am a bot' not in c.body):
                        comments.append(c.body)
                    if len(comments) >= max_response_cache:
                        break
            #Format post and comments to feed to model
            entry = '<|startoftext|>[title]'
            entry += title

            if selftext !='':
                entry += '\n[selftext]'+selftext
            else:
                entry += '\n'
            for comment in comments:
                entry += '[comment]'+comment+'\n'
            entry = entry[:-1] + '<|endoftext|>\n'
            total_entries.append(entry)
        if len(total_entries)%100 == 0:
            print(len(total_entries))
    return total_entries
    
    


    

In [ ]:
republican_data = generate_data('Republican')

In [ ]:
democrat_data = generate_data('democrats')

In [72]:
file = open("demo.txt","w",encoding="utf-8") 
file.writelines(democrat_data) 
file.close() 

In [73]:
file = open("republic.txt","w",encoding="utf-8") 
file.writelines(republican_data) 
file.close() 